In [1]:
import os

from helpers.helpers_clean import *
from helpers.helpers_difference import *
from helpers.helpers_buffer import *
from helpers.helpers_merge import *
from helpers.helpers_read_data import *


In [2]:
### Input Data 
#input_folder = "D:/2_Analytics/9_LULC_classification/automation/WF/WF"

input_folder = "D:/2_Analytics/9_LULC_classification/automation/data_2/Chirmiri OCM_Fix_Geometry_Mansi/Chirmiri OCM_Fix_Geometry_Mansi"

# output folder 
#output_folder_path = "D:/2_Analytics/9_LULC_classification/automation/output"
output_folder_path = "D:/2_Analytics/9_LULC_classification/automation/data_2/output_folder"
os.makedirs(output_folder_path, exist_ok=True)


## parameters
buffer = 10   ## in meters
min_area_removed = 25  # sq.m
inset_buffer_for_AOI = 0.2   # in meters

In [3]:
## fetch the input data 

aoi_path, settlement_path, other_shps = fetch_shapefile_paths(input_folder)

print("AOI file:", aoi_path)
print("Settlement file:", settlement_path)
print("Other shapefiles:", other_shps)

pairs = make_shapefile_pairs(other_shps)
print(pairs)

AOI file: D:\2_Analytics\9_LULC_classification\automation\data_2\Chirmiri OCM_Fix_Geometry_Mansi\Chirmiri OCM_Fix_Geometry_Mansi\aoi.shp
Settlement file: D:\2_Analytics\9_LULC_classification\automation\data_2\Chirmiri OCM_Fix_Geometry_Mansi\Chirmiri OCM_Fix_Geometry_Mansi\settlement.shp
Other shapefiles: ['D:\\2_Analytics\\9_LULC_classification\\automation\\data_2\\Chirmiri OCM_Fix_Geometry_Mansi\\Chirmiri OCM_Fix_Geometry_Mansi\\C__Agricultural_Land.shp', 'D:\\2_Analytics\\9_LULC_classification\\automation\\data_2\\Chirmiri OCM_Fix_Geometry_Mansi\\Chirmiri OCM_Fix_Geometry_Mansi\\C__Barren_Area.shp', 'D:\\2_Analytics\\9_LULC_classification\\automation\\data_2\\Chirmiri OCM_Fix_Geometry_Mansi\\Chirmiri OCM_Fix_Geometry_Mansi\\C__Boundaries.shp', 'D:\\2_Analytics\\9_LULC_classification\\automation\\data_2\\Chirmiri OCM_Fix_Geometry_Mansi\\Chirmiri OCM_Fix_Geometry_Mansi\\C__Forest_Area.shp', 'D:\\2_Analytics\\9_LULC_classification\\automation\\data_2\\Chirmiri OCM_Fix_Geometry_Mansi\\Ch

Operation 1: Get the buffers to settlement 

In [4]:
settlement_folder_path = os.path.join(output_folder_path, "related_files_settlement")
os.makedirs(settlement_folder_path, exist_ok=True)

buffered_settlement_path = os.path.join(settlement_folder_path, "buffered_settlement_shp.shp")
dissolved_settlement_path = os.path.join(settlement_folder_path, "dissolved_settlement_shp.shp")
dissolved_and_gap_filled_settlement_path = os.path.join(settlement_folder_path, "dissoved_and_gap_filled_settlement_shp.shp")


# Example usage:
gdf_buf, gdf_diss, gdf_diss_filled = buffer_and_dissolve(
    input_shp=settlement_path,
    buffer_m=buffer,
    out_buffered=buffered_settlement_path,
    out_dissolved=dissolved_settlement_path,
    out_filled_dissolved=dissolved_and_gap_filled_settlement_path ,
    min_area_remove=0
)


Loaded: D:\2_Analytics\9_LULC_classification\automation\data_2\Chirmiri OCM_Fix_Geometry_Mansi\Chirmiri OCM_Fix_Geometry_Mansi\settlement.shp
Original CRS: EPSG:32644
Created 7174 buffered geometries (buffer=10 m).
Dissolved all buffered geometries into one.
Filled holes inside dissolved geometry (interiors removed).
Saved buffered: D:/2_Analytics/9_LULC_classification/automation/data_2/output_folder\related_files_settlement\buffered_settlement_shp.shp
Saved dissolved: D:/2_Analytics/9_LULC_classification/automation/data_2/output_folder\related_files_settlement\dissolved_settlement_shp.shp
Saved filled dissolved: D:/2_Analytics/9_LULC_classification/automation/data_2/output_folder\related_files_settlement\dissoved_and_gap_filled_settlement_shp.shp


opeartion 2: subtract (all other layers) from dissolved settlement 

In [5]:
## merge all classes without dissolved settlment

classes_folder_path = os.path.join(output_folder_path, "related_files_classes")
os.makedirs(classes_folder_path, exist_ok=True)

merged_classes_without_settlement_path = os.path.join(classes_folder_path, "merged_classes_except_settlement_shp.shp")

merged_classes_without_settlement_gdf = merge_shapefiles_with_class(pairs, merged_classes_without_settlement_path, dissolve=False)



## subtract all features from dissolved settlement 


cropped_dissolved_settlement_path = os.path.join(settlement_folder_path, "cropped_dissolved_settlement_shp.shp")

cropped_dissolved_settlement_gdf = subtract_shapefiles(dissolved_and_gap_filled_settlement_path, 
                                                        merged_classes_without_settlement_path, 
                                                        cropped_dissolved_settlement_path)



exploded_path = os.path.join(settlement_folder_path, "exploded_settlement_shp.shp")
filtered_settlement_path = os.path.join(settlement_folder_path, "final_cropped_settlement_shp.shp")

exploded, filtered = explode_and_filter_features(
    cropped_dissolved_settlement_path,
    exploded_path,
    filtered_settlement_path,
    min_area_sqm=min_area_removed
)

Saved difference shapefile to: D:/2_Analytics/9_LULC_classification/automation/data_2/output_folder\related_files_settlement\cropped_dissolved_settlement_shp.shp
Area (dissolved): 1423800.43
Area (merged to subtract): 12570896.44
Area (difference/out): 1166069.08


Operation 3: subtract all features and cropped dissolved settlement from AOI 

In [ ]:
merged_classes_path = os.path.join(classes_folder_path, "merged_classes_except_unclassified.shp")
# Example usage:
merged_gdf = merge_two_shapefiles_keep_class(filtered_settlement_path, 
                                             merged_classes_without_settlement_path, 
                                             merged_classes_path)


unclassified_path = os.path.join(output_folder_path, "unclassified_shp.shp")
merged_with_unclassified_path = os.path.join(classes_folder_path, "all_classes_including_unclassified_shp.shp")
unclassified_gdf, merged_gdf = subtract_using_aoi_inset_and_write(
    aoi_shp=aoi_path,
    classes_shp=merged_classes_path,
    unclassified_shp=unclassified_path,
    merged_shp=merged_with_unclassified_path,
    inset_m=inset_buffer_for_AOI,
    reproject_classes_to_aoi=True,
    explode_result=True,
    min_area_sqm=0.1,
    dissolve_by_class=False
)


output_path = os.path.join(output_folder_path, "final_classes_shp.shp")

cleaned_gdf = clean_attributes_with_area(
    input_shp=merged_with_unclassified_path ,
    output_shp=output_path
)

Merged 45 + 351 features -> 396 written to: D:/2_Analytics/9_LULC_classification/automation/data_2/output_folder\related_files_classes\merged_classes_except_unclassified.shp
AOI inset by 0.2 m: features 1
Unclassified features: 144, total area (units²): 191522.99
Merged (clipped to AOI) features: 539, total area (units²): 13909165.12
Saved unclassified -> D:/2_Analytics/9_LULC_classification/automation/data_2/output_folder\unclassified_shp.shp
Saved merged (classes + unclassified, clipped to AOI) -> D:/2_Analytics/9_LULC_classification/automation/data_2/output_folder\related_files_classes\all_classes_including_unclassified_shp.shp
Saved cleaned shapefile to D:/2_Analytics/9_LULC_classification/automation/data_2/output_folder\final_classes_shp.shp with 539 features.
